In [1]:
import selenium
from config import *
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
#Apertura del navegador
driver = webdriver.Chrome()
driver.get(URL)

In [3]:
#Ingreso al sistema
input_username = driver.find_element(By.XPATH, '//input[@placeholder="Usuario/Cuil/Cuit"]')
input_password = driver.find_element(By.XPATH, '//input[@placeholder="Contraseña"]')
button_loggin  = driver.find_element(By.XPATH, '//button[@class="btn btn-default z-button"]')

input_username.send_keys(USERNAME)
input_password.send_keys(PASSWORD)
button_loggin.click()
sleep(3)


In [4]:
#Nueva notificación electrónica
notificaciones_electronicas = driver.find_element(By.XPATH, '//li[@title="Notificaciones Electronicas"]')
notificaciones_electronicas.click()
sleep(10)

nueva_notificacion = driver.find_element(By.XPATH, '//li[@title="Nueva Notificación"]')
nueva_notificacion.click()

In [5]:
#Función que ingresa todas las empresas.
def add_companies(COMPANIES):
    for cuit in COMPANIES.values():
        input_cuit = driver.find_element(By.XPATH, '//input[@name="cuit1"]')
        input_cuit.send_keys(cuit)
        button_search = driver.find_elements(By.XPATH, '//td[@class="z-row-inner"]//button[@class="z-button"]')[0]
        button_search.click()
        button_add   = driver.find_elements(By.XPATH, '//td[@class="z-row-inner"]//button[@class="z-button"]')[1]
        button_add.click()
        sleep(1)

In [6]:
#Ejecutar fnción add_companies.
add_companies(COMPANIES)
sleep(1)

In [7]:
#Añadiendo cuerpo
header = 'S/Faltante Prueba Python' 
body   = 'Esta es una prueba automatizada por Python'

input_referencia = driver.find_element(By.XPATH, '//div[@class="z-hlayout-inner"]//input[@class="z-textbox"]')
input_referencia.send_keys(header)

input_cuerpo     = driver.find_element(By.XPATH, '//textarea[@placeholder="Ingresar texto de la notificación"]')
input_cuerpo.send_keys(body)

button_check     = driver.find_elements(By.XPATH, '//tbody[@class="z-rows"]//tr[@class="z-row"]//td[@class="z-row-inner"]//div[@class="z-row-content"]//div[@class="z-hlayout"]//div[@class="z-hlayout-inner"]//button[@class="z-button"]')[0]
button_check.click()

#Seleccionando la opción "Quiero recibir un aviso cuando la tarea se firme"
input_aviso = driver.find_elements(By.XPATH, '//div[@class="z-window-content"]//div[@class="z-hlayout"]//div[@class="z-hlayout-inner"]//input[@type="checkbox"]')[1]
input_aviso.click()

button_send = driver.find_elements(By.XPATH, '//div[@class="z-hlayout"]//div[@class="z-hlayout-inner"]//button[@class="z-button"]')[9]
button_send.click()
sleep(2)

input_send_username = driver.find_element(By.XPATH, './/input[@placeholder="Ingrese el usuario"]')
input_send_username.send_keys('Lucas Marin( MARINL - DGFU )')
sleep(4)

button_send_notif = driver.find_element(By.XPATH, './/tbody[@class="z-rows"]//tr[@class="GridLayoutNoBorder z-row"]//td[@class="z-row-inner"]//div[@class="z-row-content"]//button[@class="z-button"]')
button_send_notif.click()
sleep(2)

button_ok =  driver.find_element(By.XPATH, './/button[@class="z-messagebox-button z-button"]')
button_ok.click()
